In [1]:
!pip install -qU openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.3/164.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00


In [7]:

from agents import (
    Agent,                           # 🤖 Core agent class
    Runner,                          # 🏃 Runs the agent
    AsyncOpenAI,                     # 🌐 OpenAI-compatible async client
    OpenAIChatCompletionsModel,     # 🧠 Chat model interface
    function_tool,                   # 🛠️ Decorator to turn Python functions into tools
    set_default_openai_client,      # ⚙️ (Optional) Set default OpenAI client
    set_tracing_disabled,           # 🚫 Disable internal tracing/logging
)

# 🌿 Load environment variables from .env file


# 🚫 Disable tracing for clean output (optional for beginners)


# 🛠️ 3) Define tools (functions wrapped for tool calling)
@function_tool
def multiply(a: int, b: int) -> int:
    """🧮 Exact multiplication (use this instead of guessing math)."""
    return a * b

@function_tool
def sum(a: int, b: int) -> int:
    """➕ Exact addition (use this instead of guessing math)."""
    return a + b

# 🤖 4) Create agent and register tools
agent: Agent = Agent(
    name="Assistant",  # 🧑‍🏫 Agent's identity
    instructions=(
        "You are a helpful assistant. "
        "Always use tools for math questions. Always follow DMAS rule (division, multiplication, addition, subtraction). "
        "Explain answers clearly and briefly for beginners."
    ),
    model="gpt-4.1",
    tools=[multiply, sum],  # 🛠️ Register tools here
)

# 🧪 5) Run the agent with a prompt (tool calling expected)
prompt = "what is 19 + 23 * 2?"
result = Runner.run_sync(agent, prompt)

# 📤 Print the final result from the agent
print("\n🤖 CALLING AGENT\n")
print(result.final_output)


🤖 CALLING AGENT

According to the DMAS rule (first multiplication, then addition):

1. Multiply 23 × 2 = 46
2. Add 19 + 46 = 65

So, 19 + 23 × 2 = 65.


In [2]:

import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
from agents import Agent, ModelSettings, Runner

# Create a precise math tutor
math_tutor = Agent(
    name="Math Tutor",
    instructions="You are a precise math tutor. Always show your work step by step.",
    model_settings=ModelSettings(
        temperature=0.1,  # Very focused
        max_tokens=500    # Enough for detailed steps
    )
)

result = Runner.run_sync(math_tutor, "Solve: 2x + 5 = 13")
print(result.final_output)

To solve the equation \(2x + 5 = 13\), follow these steps:

1. **Subtract 5 from both sides** to isolate the term with \(x\):

   \[
   2x + 5 - 5 = 13 - 5
   \]

   Simplifying both sides gives:

   \[
   2x = 8
   \]

2. **Divide both sides by 2** to solve for \(x\):

   \[
   \frac{2x}{2} = \frac{8}{2}
   \]

   Simplifying both sides gives:

   \[
   x = 4
   \]

Thus, the solution is \(x = 4\).


In [13]:
import os
import asyncio
from dotenv import load_dotenv, find_dotenv
from dataclasses import dataclass
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool, RunContextWrapper




@dataclass
class UserContext:
    username: str
    email: str | None = None

@function_tool()
async def search(local_context: RunContextWrapper[UserContext], query: str) -> str:
    import time
    time.sleep(5)  # Simulating a delay for the search operation
    return "No results found."

async def special_prompt(special_context: RunContextWrapper[UserContext], agent: Agent[UserContext]) -> str:
    # who is user?
    # which agent
    print(f"\nUser: {special_context.context},\n Agent: {agent.name}\n")
    return f"You are a math expert. User: {special_context.context.username}, Agent: {agent.name}. Please assist with math-related queries."

math_agent: Agent = Agent(name="Genius", instructions=special_prompt, model="gpt-4.1", tools=[search])
# [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]

async def call_agent():
    # Call the agent with a specific input
    user_context = UserContext(username="abdullah")

    output = await Runner.run(
        starting_agent=math_agent,
        input="search for the best math tutor in my area",
        context=user_context
        )
    print(f"\n\nOutput: {output.final_output}\n\n")

asyncio.run(call_agent())


User: UserContext(username='abdullah', email=None),
 Agent: Genius


User: UserContext(username='abdullah', email=None),
 Agent: Genius



Output: I couldn't find specific information about the best math tutor in your area. Could you please provide your location (city or ZIP code) so I can narrow down the search and provide better recommendations?




In [14]:

def main():
    """Learn Dynamic Instructions with simple examples."""
    print("🎭 Dynamic Instructions: Make Your Agent Adapt")
    print("=" * 50)

    # 🎯 Example 1: Basic Dynamic Instructions
    print("\n🎭 Example 1: Basic Dynamic Instructions")
    print("-" * 40)

    def basic_dynamic(context: RunContextWrapper, agent: Agent) -> str:
        """Basic dynamic instructions function."""
        return f"You are {agent.name}. Be helpful and friendly."

    agent_basic = Agent(
        name="Dynamic Agent",
        instructions=basic_dynamic,

    )

    result = Runner.run_sync(agent_basic, "Hello!")
    print("Basic Dynamic Agent:")
    print(result.final_output)

main()

🎭 Dynamic Instructions: Make Your Agent Adapt

🎭 Example 1: Basic Dynamic Instructions
----------------------------------------
Basic Dynamic Agent:
Hi there! How can I assist you today?


In [16]:
# 🎯 Example 2: Context-Aware Instructions


def context_aware(context: RunContextWrapper, agent: Agent) -> str:
    """Context-aware instructions based on message count."""
    message_count = len(getattr(context, 'messages', []))

    if message_count == 0:
        return "You are a welcoming assistant. Introduce yourself!"
    elif message_count < 3:
        return "You are a helpful assistant. Be encouraging and detailed."
    else:
        return "You are an experienced assistant. Be concise but thorough."

agent_context = Agent(
        name="Context Aware Agent",
        instructions=context_aware,

    )

    # Test with multiple messages
result1 = Runner.run_sync(agent_context, "Hello!")
print("First message:")
print(result1.final_output)

result2 = Runner.run_sync(agent_context, "Tell me about Python")
print("\nSecond message:")
print(result2.final_output)

First message:
Hello! I'm your friendly assistant, here to help with anything you need. How can I assist you today?

Second message:
Python is a high-level, interpreted programming language known for its clear syntax and readability. Created by Guido van Rossum and first released in 1991, Python emphasizes code readability and simplicity, making it a great choice for both beginners and experienced developers.

It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python's extensive standard library and thriving ecosystem of third-party packages make it versatile for various applications, such as web development, data analysis, machine learning, automation, and more.

Thanks to its community, Python continues to grow and evolve, maintaining its reputation as one of the most popular programming languages in the world.


In [18]:
import datetime

def time_based(context: RunContextWrapper, agent: Agent) -> str:
    """Time-based instructions based on current hour."""
    current_hour = datetime.datetime.now().hour
    print(current_hour)

    if 6 <= current_hour < 12:
        return f"You are {agent.name}. Good morning! Be energetic and positive."
    elif 12 <= current_hour < 17:
        return f"You are {agent.name}. Good afternoon! Be focused and productive."
    else:
        return f"You are {agent.name}. Good evening! Be calm and helpful."

agent_time = Agent(
        name="Time Aware Agent",
        instructions=time_based,
    )

result = Runner.run_sync(agent_time, "How are you today?")
print("Time-Based Agent:")
print(result.final_output)

17
Time-Based Agent:
Good evening! I'm here and ready to help. How can I assist you today?


In [21]:
 # 🎯 Example 4: Stateful Instructions (Remembers)
print("\n🎭 Example 4: Stateful Instructions")
print("-" * 40)

class StatefulInstructions:
    """Stateful instructions that remember interaction count."""
    def __init__(self):
        self.interaction_count = 0

    def __call__(self, context: RunContextWrapper, agent: Agent) -> str:
        self.interaction_count += 1

        if self.interaction_count == 1:
            return "You are a learning assistant. This is our first interaction - be welcoming!"
        elif self.interaction_count <= 3:
            return f"You are a learning assistant. This is interaction #{self.interaction_count} - build on our conversation."
        else:
            return f"You are an experienced assistant. We've had {self.interaction_count} interactions - be efficient."

instruction_gen = StatefulInstructions()

agent_stateful = Agent(
    name="Stateful Agent",
    instructions=instruction_gen,

)

# Test multiple interactions
for i in range(3):
    result = Runner.run_sync(agent_stateful, f"Question {i+1}: Tell me about AI")
    print(f"Interaction {i+1}:")
    print(result.final_output[:100] + "...")
    print()


🎭 Example 4: Stateful Instructions
----------------------------------------
Interaction 1:
Hello and welcome! I'd be delighted to tell you about AI. Artificial Intelligence, or AI, refers to ...

Interaction 2:
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines designed to ...

Interaction 3:
AI, or artificial intelligence, refers to the simulation of human intelligence in machines. These sy...



In [ ]:
# 🎭 Dynamic Instructions: Make Your Agent Adapt
# Simple examples to learn dynamic instructions

import os
from dotenv import load_dotenv
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, RunContextWrapper




def main():
    """Learn Dynamic Instructions with simple examples."""
    print("🎭 Dynamic Instructions: Make Your Agent Adapt")
    print("=" * 50)

    # 🎯 Example 1: Basic Dynamic Instructions
    print("\n🎭 Example 1: Basic Dynamic Instructions")
    print("-" * 40)

    def basic_dynamic(context: RunContextWrapper, agent: Agent) -> str:
        """Basic dynamic instructions function."""
        return f"You are {agent.name}. Be helpful and friendly."

    agent_basic = Agent(
        name="Dynamic Agent",
        instructions=basic_dynamic,

    )

    result = Runner.run_sync(agent_basic, "Hello!")
    print("Basic Dynamic Agent:")
    print(result.final_output)

    # 🎯 Example 2: Context-Aware Instructions
    print("\n🎭 Example 2: Context-Aware Instructions")
    print("-" * 40)

    def context_aware(context: RunContextWrapper, agent: Agent) -> str:
        """Context-aware instructions based on message count."""
        message_count = len(getattr(context, 'messages', []))

        if message_count == 0:
            return "You are a welcoming assistant. Introduce yourself!"
        elif message_count < 3:
            return "You are a helpful assistant. Be encouraging and detailed."
        else:
            return "You are an experienced assistant. Be concise but thorough."

    agent_context = Agent(
        name="Context Aware Agent",
        instructions=context_aware,
        model=model
    )

    # Test with multiple messages
    result1 = Runner.run_sync(agent_context, "Hello!")
    print("First message:")
    print(result1.final_output)

    result2 = Runner.run_sync(agent_context, "Tell me about Python")
    print("\nSecond message:")
    print(result2.final_output)

    # 🎯 Example 3: Time-Based Instructions
    print("\n🎭 Example 3: Time-Based Instructions")
    print("-" * 40)

    import datetime

    def time_based(context: RunContextWrapper, agent: Agent) -> str:
        """Time-based instructions based on current hour."""
        current_hour = datetime.datetime.now().hour

        if 6 <= current_hour < 12:
            return f"You are {agent.name}. Good morning! Be energetic and positive."
        elif 12 <= current_hour < 17:
            return f"You are {agent.name}. Good afternoon! Be focused and productive."
        else:
            return f"You are {agent.name}. Good evening! Be calm and helpful."

    agent_time = Agent(
        name="Time Aware Agent",
        instructions=time_based,
        model=model
    )

    result = Runner.run_sync(agent_time, "How are you today?")
    print("Time-Based Agent:")
    print(result.final_output)

    # 🎯 Example 4: Stateful Instructions (Remembers)
    print("\n🎭 Example 4: Stateful Instructions")
    print("-" * 40)

    class StatefulInstructions:
        """Stateful instructions that remember interaction count."""
        def __init__(self):
            self.interaction_count = 0

        def __call__(self, context: RunContextWrapper, agent: Agent) -> str:
            self.interaction_count += 1

            if self.interaction_count == 1:
                return "You are a learning assistant. This is our first interaction - be welcoming!"
            elif self.interaction_count <= 3:
                return f"You are a learning assistant. This is interaction #{self.interaction_count} - build on our conversation."
            else:
                return f"You are an experienced assistant. We've had {self.interaction_count} interactions - be efficient."

    instruction_gen = StatefulInstructions()

    agent_stateful = Agent(
        name="Stateful Agent",
        instructions=instruction_gen,
        model=model
    )

    # Test multiple interactions
    for i in range(3):
        result = Runner.run_sync(agent_stateful, f"Question {i+1}: Tell me about AI")
        print(f"Interaction {i+1}:")
        print(result.final_output[:100] + "...")
        print()

    # 🎯 Example 5: Exploring Context and Agent
    print("\n🎭 Example 5: Exploring Context and Agent")
    print("-" * 40)

    def explore_context_and_agent(context: RunContextWrapper, agent: Agent) -> str:
        """Explore what's available in context and agent."""
        # Access conversation messages
        messages = getattr(context, 'messages', [])
        message_count = len(messages)

        # Access agent properties
        agent_name = agent.name
        tool_count = len(agent.tools)

        return f"""You are {agent_name} with {tool_count} tools.
        This is message #{message_count} in our conversation.
        Be helpful and informative!"""

    agent_explorer = Agent(
        name="Context Explorer",
        instructions=explore_context_and_agent,
        model=model
    )

    result = Runner.run_sync(agent_explorer, "What can you tell me about yourself?")
    print("Context Explorer Agent:")
    print(result.final_output)

    print("\n🎉 You've learned Dynamic Instructions!")
    print("💡 Try changing the functions and see what happens!")

if __name__ == "__main__":
    main()

In [24]:
print("\n🎭 Example 5: Exploring Context and Agent")
print("-" * 40)

def explore_context_and_agent(context: RunContextWrapper, agent: Agent) -> str:
  """Explore what's available in context and agent."""
  # Access conversation messages
  messages = getattr(context, 'messages', [])
  message_count = len(messages)

  # Access agent properties
  agent_name = agent.name
  tool_count = len(agent.tools)

  return f"""You are {agent_name} with {tool_count} tools.
  This is message #{message_count} in our conversation.
  Be helpful and informative!"""

agent_explorer = Agent(
  name="Context Explorer",
  instructions=explore_context_and_agent,
)

result = Runner.run_sync(agent_explorer, "What can you tell me about yourself?")
print("Context Explorer Agent:")
print(result.final_output)

print("\n🎉 You've learned Dynamic Instructions!")
print("💡 Try changing the functions and see what happens!")


main()


🎭 Example 5: Exploring Context and Agent
----------------------------------------
Context Explorer Agent:
Sure! I'm Context Explorer, here to help answer your questions and provide information on a wide range of topics. My goal is to assist you with clear, concise, and accurate responses. How can I help you today?

🎉 You've learned Dynamic Instructions!
💡 Try changing the functions and see what happens!
🎭 Dynamic Instructions: Make Your Agent Adapt

🎭 Example 1: Basic Dynamic Instructions
----------------------------------------
Basic Dynamic Agent:
Hi there! How can I assist you today?


In [25]:
from agents import Agent, Runner, trace
import asyncio

from agents import Agent, Runner, trace

async def main():
    agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

    with trace("Joke workflow"):
        first_result = await Runner.run(agent, "Tell me a joke")
        second_result = await Runner.run(agent, f"Rate this joke: {first_result.final_output}")
        print(f"Joke: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

asyncio.run(main())

Joke: Why don't scientists trust atoms?

Because they make up everything!
Rating: That's a classic! I'd give it an 8 out of 10. It's clever and scientific humor always gets a good chuckle.
